In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [3]:
url = "https://raw.githubusercontent.com/fja05680/sp500/refs/heads/master/S%26P%20500%20Historical%20Components%20%26%20Changes(07-12-2025).csv"
df = pd.read_csv(url)
df['date'] = pd.to_datetime(df['date'])
dates = pd.date_range(df['date'].iloc[0], df['date'].iloc[-1])
df = df.set_index("date")
# añadimos fechas faltantes, y rellenamos con los anterior
df = df.reindex(dates)
df.tickers = df.tickers.ffill() #TODO: ffill o bfill?
df.head()

,tickers
1996-01-02,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1996-01-03,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1996-01-04,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1996-01-05,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1996-01-06,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."


## Problemas a investigar

Los datos de las empresas del SP&500 es algo inconsistente por ejemplo, segun [wikipedia](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Selected_changes_to_the_list_of_S&P_500_components), ACE es sustituido el 19 de Enero de 2016 por EXR, pero comprobando, ACE ni si quiera esta en la lista. Esto sucede para mas casos.

In [4]:
date = pd.to_datetime("2016-01-15")
date2 = pd.to_datetime("2016-01-25")

removed_com = 'ACE'
new_com = 'EXR'
print(f"{removed_com} in {date}? {removed_com in (df.loc[date].tickers)}")
print(f"{removed_com} in {date2}? {removed_com in (df.loc[date2].tickers)}")
print(f"{new_com} in {date}? {new_com in (df.loc[date].tickers)}")
print(f"{new_com} in {date2}? {new_com in (df.loc[date2].tickers)}")

tickers = df.loc[date].tickers
tickers2 = df.loc[date2].tickers

companies = set(tickers.split(","))
companies2 = set(tickers2.split(","))

print(len(companies))
print(len(companies2))
print(companies^companies2)

ACE in 2016-01-15 00:00:00? False
ACE in 2016-01-25 00:00:00? False
EXR in 2016-01-15 00:00:00? False
EXR in 2016-01-25 00:00:00? True
504
505
{'EXR'}


O incluso ni se aplica correctamente, por ejemplo aqui, donde el 5 de Diciembre del 2000, se cambio OI por SBL, pero el dataset no lo refleja, añade RHI

In [5]:
date = pd.to_datetime("2000-12-1")
date2 = pd.to_datetime("2000-12-6")

removed_com = 'OI'
new_com = 'SBL'
print(f"{removed_com} in {date}? {removed_com in (df.loc[date].tickers)}")
print(f"{removed_com} in {date2}? {removed_com in (df.loc[date2].tickers)}")
print(f"{new_com} in {date}? {new_com in (df.loc[date].tickers)}")
print(f"{new_com} in {date2}? {new_com in (df.loc[date2].tickers)}")

tickers = df.loc[date].tickers
tickers2 = df.loc[date2].tickers

companies = set(tickers.split(","))
companies2 = set(tickers2.split(","))

print(len(companies))
print(len(companies2))
print(companies^companies2)

OI in 2000-12-01 00:00:00? True
OI in 2000-12-06 00:00:00? True
SBL in 2000-12-01 00:00:00? False
SBL in 2000-12-06 00:00:00? False
491
492
{'RHI'}


Otro problema es el cambio de tickers por fusion (merge). Esta [lista](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Selected_changes_to_the_list_of_S&P_500_components) muestra como algunos tickers existian en el pasado pero fueron sustituidos por nuevos. Por ejemplo, el 6 de Julio de 2015, Kraft fue adherido por Heinz, cambiando el simbolo de KRFT a KHC.

In [6]:
date = pd.to_datetime("2015-7-4")
date2 = pd.to_datetime("2015-7-7")

tickers = df.loc[date].tickers
tickers2 = df.loc[date2].tickers

print(f"KRFT in {date}? {'KRFT' in (df.loc[date].tickers)}")
print(f"KRFT in {date2}? {'KRFT' in (df.loc[date2].tickers)}")

print(f"KHC in {date}? {'KHC' in (df.loc[date].tickers)}")
print(f"KHC in {date2}? {'KHC' in (df.loc[date2].tickers)}")



KRFT in 2015-07-04 00:00:00? True
KRFT in 2015-07-07 00:00:00? False
KHC in 2015-07-04 00:00:00? False
KHC in 2015-07-07 00:00:00? True


Esto trae consigo otro problema, al dejar de existir un simbolo, yfinance lo elimina. Con el ejemplo anterior, perdemos absolutamente toda la historia de KRFT.

In [7]:
date2 = pd.to_datetime("2025-1-7")

yf.download("KRFT", end=date2, auto_adjust=True)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['KRFT']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-12-07 -> 2025-01-07 00:00:00)')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,KRFT,KRFT,KRFT,KRFT,KRFT,KRFT
Date,,,,,,


## Conclusion errores

- Inconsistencias en los cambios de empresas/tickers, pues solo se realizan parcialmente
- Cambios en el dataset pueden ser por distintos motivos (susticion, compra, merge, etc.)
- Al dejar de existir algunos tickers, se pierden todos sus datos pasados

## Soluciones posibles

- Respecto a las incosistencias, la pagina de la wikipedia parece bastante fiable. Te dan las 503 empresas actuales y todos los cambios hasta el July 1, 1976. Alomejor tiene sentido scrapear esta informacion y partiendo de las empresas iniciales ir creando el dataset hacia atras. (facil)
- Si se hace el scrapeo, tratar de categorizar la decision del cambio para mejorar el tratamiento posterior. (medio)
- Respecto a la perdida de informacion de las empresas que son adquiridas o fusionadas, alomejor buscar otro tipo de API's o datos en internet. (dificil)


In [8]:
df.tickers = df.tickers.str.split(",")
unique_tickers = df.explode("tickers").tickers.unique()
unique_tickers

array(['AAL', 'AAMRQ', 'AAPL', ..., 'WSM', 'COIN', 'DDOG'],
      shape=(1181,), dtype=object)

In [9]:
# vamos guardando objetos de los que podremos obtener dividendos y splits posteriormente
# tambien nos sirve para ver los tickers que dan algun tipo de problema

# comentado porque es ineficiente y largo

# div_mapper = {}
# split_mapper = {}
# for tick in unique_tickers:
#     obj = yf.Ticker(tick)
#     div_mapper[tick] = obj.dividends
#     split_mapper[tick] = obj.splits


In [10]:
# ejemplo de creacion
# iteramos en orden inverso para mas exito de descarga
filas = []
# def get_today_dividends(ticker, date):
#     if not div_mapper[ticker].empty:
#         return div_mapper[ticker].loc[div_mapper[ticker].index.date == date.date()].sum()
#     else:
#         return 0.0

for i, date in enumerate(df.index[::-1]):
    end_date = date + pd.Timedelta(days=1)
    
    data = yf.download(df.loc[date].tickers, start=date, end=end_date, auto_adjust=True, interval='1d')
    data = data.stack(level=1, future_stack=True).reset_index()
    data = data.rename(columns={'level_1': 'ticker'})
    # data['dividends'] = get_today_dividends(data['Ticker'], date)

    filas.append(data)
    if i>1:
        break

final_data = pd.concat(filas, ignore_index=True)

[**********            20%                       ]  99 of 503 completedHTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: HES"}}}
[*********************100%***********************]  503 of 503 completed

5 Failed downloads:
['BRK.B', 'HES', 'ANSS', 'PARA']: YFTzMissingError('possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-07-09 00:00:00 -> 2025-07-10 00:00:00)')
[*********************100%***********************]  503 of 503 completed

6 Failed downloads:
['JNPR', 'BRK.B', 'HES', 'ANSS', 'PARA']: YFTzMissingError('possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-07-08 00:00:00 -> 2025-07-09 00:00:00)')
[*********************100%***********************]  503 of 503 completed

6 Failed downloads:
['JNPR', 'BRK.B', 'HES', 'ANSS', 'PARA']: YFTzMissingError('possibly delisted; no timez

In [14]:
final_data.sample(5)

Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
797,2025-07-08,LYV,NaN,144.429993,148.039993,142.240005,147.360001,2020500.0
818,2025-07-08,MO,NaN,58.593079,59.321308,58.288007,59.045762,7939200.0
1403,2025-07-07,ROK,NaN,337.632812,346.269643,336.965394,344.406805,1445100.0
1158,2025-07-07,EA,NaN,153.360626,155.987730,153.160850,155.548213,2502800.0
253,2025-07-09,ITW,NaN,258.150055,258.646975,255.208296,257.136349,866300.0
